In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Prepare people data frame
df_people = pd.read_csv('Data/demog.csv')
print(df_people.shape)
# Keep only people for which we have age
df_people = df_people[~pd.isna(df_people.age)]
print(df_people.shape)
# Target teenagers
df_people['target_age'] = df_people.age >= 50
#df_people = df_people[['userid', 'target_age', 'age']]
df_people.userid = df_people.userid.astype('category')
df_people.head()

(4282857, 15)
(1573876, 15)


,userid,gender,birthday,age,relationship_status,interested_in,mf_relationship,mf_dating,mf_random,mf_friendship,mf_whatever,mf_networking,locale,network_size,timezone,target_age
6,631f821671a94f991413e58c2b393b29,1.0,1982-08-09,30.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en_US,NaN,NaN,False
7,bdeb2156021c5da9237f5d2667ab590a,1.0,1983-07-26,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en_US,681.0,-5.0,False
12,d9fb05627d4659484703e5827690b9f9,0.0,1985-01-21,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en_US,586.0,-4.0,False
23,adc399722fccda28049cb26c5a795b16,1.0,1983-12-19,29.0,2.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,en_US,300.0,-5.0,False
29,18f92f9c1f86b5cd692e8fcc19721b1d,1.0,1985-02-19,27.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,en_US,532.0,-7.0,False


In [2]:
# Prepare likes data frame
df_likes = pd.read_csv('Data/likes.csv', header=None, names=['userid', 'like_id'])
df_likes.userid = df_likes.userid.astype('category')
print(df_likes.shape)

(55775701, 2)


In [3]:
# Prune to keep only pages with more than 1,000 users and users with at least one like
change = True
while change:
    # Keep only likes of people for which we have information
    print("Likes before and after")
    val = df_likes.shape[0]
    print(df_likes.shape)
    df_likes = df_likes[df_likes.userid.isin(df_people.userid)]
    print(df_likes.shape)
    change = val != df_likes.shape[0]

    # Keep only people with at least one like
    print("People before and after")
    val = df_people.shape[0]
    print(df_people.shape)
    df_people = df_people[df_people.userid.isin(df_likes.userid.unique())]
    print(df_people.shape)
    change = (val != df_people.shape[0]) | change
    # Keep only pages with at least 1,000 likes
    print("Pages before and after")
    counts = df_likes.like_id.value_counts()
    val = len(counts)
    high_counts = counts[counts >= 1000]
    change = (val != len(high_counts)) | change
    print(len(counts))
    df_likes = df_likes[df_likes.like_id.isin(high_counts.index.values)]
    print(len(high_counts))

Likes before and after
(55775701, 2)
(37686388, 2)
People before and after
(1573876, 16)
(588610, 16)
Pages before and after
15914
10822
Likes before and after
(33873012, 2)
(33873012, 2)
People before and after
(588610, 16)
(587745, 16)
Pages before and after
10822
10822
Likes before and after
(33873012, 2)
(33873012, 2)
People before and after
(587745, 16)
(587745, 16)
Pages before and after
10822
10822


In [4]:
# Get labels
labels = df_people.set_index('userid').loc[sorted(df_likes.userid.unique())].target_age.values
ages = df_people.set_index('userid').loc[sorted(df_likes.userid.unique())].age.values
# Build sparse matrix
users_c = np.array(sorted(df_likes.userid.unique()))
pages_c = np.array(sorted(df_likes.like_id.unique()))
row = pd.Categorical(df_likes.userid, categories=users_c, ordered=True).codes
col = pd.Categorical(df_likes.like_id, categories=pages_c, ordered=True).codes
sparse_mat = csr_matrix((np.ones(df_likes.shape[0]), (row, col)), shape=(users_c.size, pages_c.size))
sparse_mat

<587745x10822 sparse matrix of type '<class 'numpy.float64'>'
	with 33873012 stored elements in Compressed Sparse Row format>

In [5]:
# Build machine learning model
X_train, X_test, y_train, y_test, age_train, age_test = train_test_split(sparse_mat, labels, ages,
                                                                         test_size=0.3, random_state=42)

model = LogisticRegression(random_state=42, solver='lbfgs').fit(X_train, y_train)
fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:, 1])
metrics.auc(fpr, tpr)

C:\Users\ferlo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8574013931369411

In [6]:
# Load names of facebook pages
df_names = pd.read_csv('Data/fb_like.csv', encoding='latin-1', error_bad_lines=False, warn_bad_lines=False)
print(df_names.shape)
df_names.head()
df_names = df_names[df_names.like_id.isin(pages_c)]
print(df_names.shape)
df_names = df_names.set_index('like_id')
df_names.head()

(19041200, 2)
(10821, 2)


,name
like_id,
9254817079,photography
103144503058854,Poetry
104046862964234,Writing
105667052799444,Piano
106059522759137,English


In [7]:
# Get threshold
probs = model.predict_proba(X_test)[:, 1]
threshold = np.percentile(probs, 99)
print(threshold)
# Keep only people with probabilities above threshold
top_x = X_test[probs > threshold, :]
top_y = y_test[probs > threshold]
top_age = age_test[probs > threshold]
# Take a look at average number of likes
print(top_x.sum(axis=1).mean())
print(np.median(np.array(top_x.sum(axis=1))))
# Keep interesting errors
top_x = top_x[top_age >= 30]
top_y = top_y[top_age >= 30]
top_age = top_age[top_age >= 30]
top_x = top_x[top_age <= 40]
top_y = top_y[top_age <= 40]
top_age = top_age[top_age <= 40]
print("Errors of people between 30 and 40 years")
print(top_x.shape[0])
# Take a look at probabilities
print(model.predict_proba(top_x)[:, 1])
print(top_x.sum(axis=1).reshape(1, -1))
order = np.array(top_x.sum(axis=1)).reshape(-1).argsort()[::-1]
top_x = top_x[order]
top_y = top_y[order]
top_age = top_age[order]

0.41116640934620974
73.29024943310658
41.0
Errors of people between 30 and 40 years
165
[0.84565248 0.74256652 0.78119628 0.72119612 0.41385715 0.68499266
 0.45898055 0.98829092 0.80072783 0.43934153 0.81048513 0.73170057
 0.58773163 0.41981386 0.74491212 0.55494788 0.47974449 0.50910381
 0.47437852 0.58547422 0.53424198 0.49039252 0.69717416 0.8333557
 0.75581428 0.44579571 0.56518143 0.92140531 0.41324849 0.42370999
 0.461326   0.48154155 0.74343522 0.57035595 0.84301434 0.51596466
 0.57772989 0.60031359 0.42048403 0.41358187 0.44502888 0.62397304
 0.65683409 0.86094985 0.68844824 0.41694971 0.99615006 0.77394067
 0.5744485  0.76198515 0.5085968  0.5547998  0.76078767 0.51821855
 0.53853298 0.75322157 0.74039866 0.88581494 0.49459344 0.98338893
 0.46801608 0.6121224  0.98678264 0.81081969 0.46479025 0.53228398
 0.49764876 0.45280455 0.71262847 0.56081876 0.83838766 0.43101128
 0.44395183 0.53432619 0.42486273 0.72832017 0.43929659 0.72226813
 0.62772289 0.46752633 0.60252258 0.448278

In [8]:
np.percentile(np.array(X_test[probs > threshold, :].sum(axis=1)).flatten(), 67)

64.0

In [10]:
import shap

# Candidates 12, 3, 15, 21
# Candidates 32, 74, 88, 39, 41, 46

def make_decision(data):
    return (model.predict_proba(data)[:, 1] > threshold).astype(float)

#for i in range(30, 99):
i = 37
print("----------------------------------------------")
print("Position: ", i)
top_x_guy = top_x[np.full(5, i), :]
print(top_x_guy[0, :].sum())
print(top_age[i])
print(model.predict_proba(top_x_guy)[0, 1])
np.random.seed(0)
default = csr_matrix(np.zeros(X_test.shape[1]))
shap_explainer = shap.KernelExplainer(make_decision, default)
shap_values = shap_explainer.shap_values(top_x_guy, nsamples=4100, l1_reg='aic')

# Top 3 predictive features are different every time. 
for i in range(shap_values.shape[0]):
    i_vals = (-shap_values[i, :]).argsort()
    print(df_names.loc[pages_c[i_vals[:3]]].name)
    print(shap_values[i, i_vals[:3]])

----------------------------------------------
Position:  37
64.0
34.0
0.41962539207798977


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.94s/it]


like_id
7284978791       ELVIS PRESLEY
182736663312    Paul McCartney
21931600316         Neil Young
Name: name, dtype: object
[0.15816004 0.14952443 0.14929379]
like_id
182736663312    Paul McCartney
21931600316         Neil Young
55555550744     Brain Pickings
Name: name, dtype: object
[0.15839785 0.1552504  0.14245585]
like_id
182736663312    Paul McCartney
21931600316         Neil Young
55555550744     Brain Pickings
Name: name, dtype: object
[0.16662192 0.15264056 0.14765539]
like_id
182736663312    Paul McCartney
38026784643      Leonard Cohen
7284978791       ELVIS PRESLEY
Name: name, dtype: object
[0.15766498 0.15552602 0.14593034]
like_id
182736663312       Paul McCartney
7401700249      Bruce Springsteen
7284978791          ELVIS PRESLEY
Name: name, dtype: object
[0.17209662 0.14778275 0.1358186 ]


In [33]:
# See how long it takes for our method to find one explanation
import explainer
import time

def scoring_function(data):
    return model.predict_proba(data)[:, 1]

# Get evidence-based explanations
explain = explainer.Explainer(scoring_function, default, prune=False)
t0 = time.time()
explanations = explain.explain(top_x_guy[0, :], thresholds=float(threshold), max_ite=50)
t1 = time.time()
print(t1-t0)
print("Explanations found")
print(len(explanations[0]))
print("Length of explanation")
print(len(explanations[0][0]))
print("First explanation")
print(df_names.loc[pages_c[explanations[0][0]]])
print(df_names.loc[pages_c[explanations[0][1]]])
print(df_names.loc[pages_c[explanations[0][2]]])
print(df_names.loc[pages_c[explanations[0][3]]])
print(df_names.loc[pages_c[explanations[0][4]]])

0.4955103397369385
Explanations found
33
Length of explanation
1
First explanation
                        name
like_id                     
182736663312  Paul McCartney
                     name
like_id                  
7284978791  ELVIS PRESLEY
                   name
like_id                
21931600316  Neil Young
                      name
like_id                   
38026784643  Leonard Cohen
                       name
like_id                    
55555550744  Brain Pickings


In [15]:
from functools import reduce

sample_size = 500
np.random.seed(0)
# Get sample of targeted observations to explain
X_targeted = X_test[probs > threshold, :]
X_sample = X_targeted[np.random.choice(X_targeted.shape[0], sample_size, replace=False), :]
X_to_explain = X_sample[np.arange(X_sample.shape[0]).repeat(5), :]
# Obtain their SHAP values
shap_values = shap_explainer.shap_values(X_to_explain, nsamples=4100, l1_reg='aic')
np.save("Data/back_up_shap_values.npy", shap_values)
#shap_values = np.load("Data/back_up_shap_values(old people).npy")
# Find top 5 most informative features
most_informative_f = np.argsort(shap_values, axis=1)[:, -3:]
# Compute number of matches
sample_matches = list()
for i in range(sample_size):
    matches = reduce(np.intersect1d, most_informative_f[(i*5):((i+1)*5), :]).size
    #matches = np.unique(most_informative_f[(i*5):((i+1)*5), :]).size
    sample_matches.append(matches)
    
print(sample_matches)

 24%|██████████████████▉                                                            | 600/2500 [12:16<43:32,  1.38s/it]C:\Users\ferlo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:604: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 8 iterations, alpha=9.357e-05, previous alpha=5.684e-05, with an active set of 9 regressors.
  ConvergenceWarning)
 24%|██████████████████▉                                                            | 601/2500 [12:16<31:52,  1.01s/it]C:\Users\ferlo\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:604: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 8 iterations, alpha=9.357e-05, previous alpha=5.684e-05, with an active set of 9 regressors.
  ConvergenceWarning)
 24%|███████████████████                                                            | 602/

[1, 3, 3, 1, 3, 2, 2, 2, 0, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2, 1, 2, 3, 2, 3, 3, 2, 3, 0, 3, 3, 1, 1, 1, 0, 0, 3, 1, 1, 1, 2, 3, 3, 2, 2, 1, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 0, 1, 3, 2, 3, 1, 3, 3, 3, 1, 2, 2, 1, 2, 2, 2, 3, 3, 2, 1, 3, 2, 3, 1, 3, 1, 3, 3, 2, 0, 3, 3, 3, 2, 1, 3, 0, 2, 1, 3, 2, 2, 3, 3, 2, 0, 3, 3, 3, 2, 0, 2, 2, 1, 3, 2, 2, 3, 0, 0, 3, 3, 1, 3, 1, 2, 2, 2, 3, 1, 1, 2, 1, 3, 3, 2, 0, 2, 3, 1, 1, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 2, 1, 0, 2, 1, 1, 3, 3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 1, 3, 3, 2, 2, 1, 2, 3, 2, 3, 3, 3, 3, 3, 3, 2, 0, 0, 3, 3, 2, 1, 3, 1, 2, 1, 2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 1, 3, 3, 1, 2, 2, 3, 1, 1, 2, 2, 0, 3, 1, 1, 2, 3, 2, 3, 3, 2, 2, 0, 3, 2, 3, 1, 3, 3, 2, 3, 2, 2, 1, 3, 2, 3, 3, 1, 1, 1, 1, 0, 2, 1, 3, 0, 3, 0, 3, 1, 2, 3, 2, 1, 1, 3, 3, 2, 1, 3, 3, 2, 3, 0, 2, 2, 2, 3, 3, 3, 3, 3, 0, 2, 1, 2, 1, 3, 2, 2, 3, 0, 3, 2, 0, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1, 3, 3, 0, 3, 2, 2, 1, 1, 0, 3, 2, 1, 2, 3, 1, 1, 2, 2, 2, 3, 1, 3, 2, 1, 1, 2, 1, 1, 

In [17]:
likes_count = np.array(X_sample.sum(axis=1)).flatten()
cut_points = list()
for i in range(1, 5):
    cut_points.append(np.percentile(likes_count, i*20))
cut_points.append(np.inf)

min_likes = 0
sample_matches = np.array(sample_matches) 
for cp in cut_points:
    print("Upper limit: ", cp)
    selected_obs = (likes_count >= min_likes) & (likes_count < cp)
    print(sample_matches[selected_obs].mean())
    min_likes = cp

Upper limit:  18.0
2.7065217391304346
Upper limit:  34.0
2.2547169811320753
Upper limit:  58.39999999999998
2.1372549019607843
Upper limit:  99.0
1.7142857142857142
Upper limit:  inf
1.1470588235294117


In [18]:
# Get counterfactual explanations
explainer.Explainer(scoring_function, default, prune=False)

t0 = time.time()
sample_e = explain.explain(X_sample, thresholds=float(threshold), max_ite=50, stop_at_first=True)
t1 = time.time()
print(t1-t0)

sample_lens = np.array([len(e[0]) for e in sample_e])
min_likes = 0
for cp in cut_points:
    print("Upper limit: ", cp)
    selected_obs = (likes_count >= min_likes) & (likes_count < cp)
    print(sample_lens[selected_obs].mean())
    min_likes = cp

15.054933071136475
Upper limit:  18.0
1.173913043478261
Upper limit:  34.0
2.0
Upper limit:  58.39999999999998
2.303921568627451
Upper limit:  99.0
3.6020408163265305
Upper limit:  inf
4.901960784313726


In [34]:
# Check out real quantiles
likes_count = X_targeted.sum(axis=1)

print(np.percentile(likes_count, 20))
print(np.percentile(likes_count, 40))
print(np.percentile(likes_count, 60))
print(np.percentile(likes_count, 80))

19.0
33.0
53.0
94.0


In [42]:
from scipy.stats import norm

# Define Cost Function
likes_per_page = np.array(sparse_mat.sum(axis=0)).reshape(-1)
costs = norm.cdf((np.mean(likes_per_page) - likes_per_page) / np.std(likes_per_page))
costs += np.full(X_train.shape[1], 0.1)

def likes_cost_func(original_obs, new_obs):
    return (original_obs - new_obs).dot(costs)

# Method to print explanation details
def print_explanation(e):
    for p in e:
        print(df_names.loc[pages_c[p]].values[0], "| Likes :", int(likes_per_page[p]))

#  Find explanations with/without costs
# Good candidates: 243, 457, 11, 148, 194
ixs = [194, 148, 457]
ixs = [11, ]
t = float(threshold)
for i in range(138, 500):
    # With cost
    cost_e = explain.explain(X_sample[i, :], thresholds=t, max_ite=300, stop_at_first=True,
                             cost_func=likes_cost_func)[0]
    cost_e = cost_e[0] if len(cost_e) > 0 else []
    # No cost
    no_cost_e = explain.explain(X_sample[i, :], thresholds=t, max_ite=300, stop_at_first=True)[0][0]
    print("ID: ", i)
    print("Explanation WITHOUT costs")
    print_explanation(no_cost_e)
    print("---")
    print("Explanation WITH costs")
    print_explanation(cost_e)
    print("*************************************************")

ID:  138
Explanation WITHOUT costs
Luther Vandross | Likes : 1105
JetBlue Airways | Likes : 1037
I SUPPORT MY PRESIDENT | Likes : 2042
---
Explanation WITH costs
*************************************************
ID:  139
Explanation WITHOUT costs
The Doctors | Likes : 1157
The Animal Rescue Site | Likes : 5664
Sarah Palin | Likes : 4552
Proud to be an American | Likes : 3938
RightChange | Likes : 2427
---
Explanation WITH costs
Best Buy | Likes : 9295
The Animal Rescue Site | Likes : 5664
Sarah Palin | Likes : 4552
Two and a Half Men | Likes : 28055
American Idol | Likes : 15792
Proud to be an American | Likes : 3938
*************************************************
ID:  140
Explanation WITHOUT costs
RightChange | Likes : 2427
---
Explanation WITH costs
Classical | Likes : 8632
*************************************************
ID:  141
Explanation WITHOUT costs
Sour Patch Kids | Likes : 4241
---
Explanation WITH costs
American Idol | Likes : 15792
Sour Gummy Worms | Likes : 13077
Starb

Dancing With The Stars | Likes : 5379
Ghost | Likes : 1932
Democrats | Likes : 1656
Joni Mitchell | Likes : 1472
---
Explanation WITH costs
Reading | Likes : 47288
Pink Floyd | Likes : 43045
Star Trek | Likes : 11683
Dancing With The Stars | Likes : 5379
The Ellen DeGeneres Show | Likes : 16944
The Godfather | Likes : 12568
American Idol | Likes : 15792
Avatar | Likes : 14592
The Lord of the Rings Trilogy (Official Page) | Likes : 14088
Movies | Likes : 22109
*************************************************
ID:  189
Explanation WITHOUT costs
I thought it was just me, until I saw someone created a group for it. | Likes : 1696
I try not to laugh during serious moments, but end up laughing anyway. | Likes : 2036
Wait, how'd you find out? Faceboook told me. | Likes : 2195
---
Explanation WITH costs
Two and a Half Men | Likes : 28055
Wait, how'd you find out? Faceboook told me. | Likes : 2195
looking through a textbook and pointing at pictures going ''thats you'' | Likes : 6123
I HATE WHEN

Stavros Flatley | Likes : 2675
*************************************************
ID:  246
Explanation WITHOUT costs
ELVIS PRESLEY | Likes : 6531
The Animal Rescue Site | Likes : 5664
The Rachel Maddow Show | Likes : 2403
Betty White | Likes : 1180
Dr. Mehmet Oz | Likes : 1978
Susan Boyle | Likes : 4230
Life's not a garden, Don't be a hoe | Likes : 2999
Paul McCartney | Likes : 2494
Join the Coffee Party Movement | Likes : 1513
Comedies | Likes : 1794
Biographies | Likes : 1285
---
Explanation WITH costs
ELVIS PRESLEY | Likes : 6531
The Daily Show | Likes : 16634
Eric Clapton | Likes : 10373
Queen | Likes : 19490
The Animal Rescue Site | Likes : 5664
Michelle Obama | Likes : 9722
The Rachel Maddow Show | Likes : 2403
Susan Boyle | Likes : 4230
Paul McCartney | Likes : 2494
Join the Coffee Party Movement | Likes : 1513
Biographies | Likes : 1285
Turn Facebook Pink For 1 Week For Breast Cancer Awareness | Likes : 12942
*************************************************
ID:  247
Explanation

THEY'RE going THERE with THEIR friends. It's called grammar, use it. | Likes : 1674
The Sound Of Music | Likes : 3980
Horses | Likes : 2656
Bridesmaids | Likes : 1306
---
Explanation WITH costs
Cold Stone Creamery | Likes : 4629
The Daily Show | Likes : 16634
Gandhi | Likes : 1121
Walking | Likes : 6441
West Side Story | Likes : 1449
The Animal Rescue Site | Likes : 5664
Democrats | Likes : 1656
Nemo | Likes : 3920
The Rachel Maddow Show | Likes : 2403
Leonard Cohen | Likes : 3625
Fawlty Towers | Likes : 1826
Gardening | Likes : 4089
Pee-wee Herman | Likes : 2312
The Good Wife | Likes : 1073
Paul McCartney | Likes : 2494
I dont care about your farm, or your fish, or your park, or your mafia!!! | Likes : 8992
Heinz Ketchup | Likes : 1619
The Sound Of Music | Likes : 3980
Horses | Likes : 2656
*************************************************
ID:  294
Explanation WITHOUT costs
Steampunk | Likes : 1653
Soul music | Likes : 1634
---
Explanation WITH costs
Queen | Likes : 19490
The Godfathe

*************************************************
ID:  334
Explanation WITHOUT costs
Nick and Norahs Infinite Playlist | Likes : 1336
---
Explanation WITH costs
Eric Clapton | Likes : 10373
Queen | Likes : 19490
*************************************************
ID:  335
Explanation WITHOUT costs
Democrats | Likes : 1656
The Rachel Maddow Show | Likes : 2403
Minnesota Twins | Likes : 1097
Pee-wee Herman | Likes : 2312
Join the Coffee Party Movement | Likes : 1513
---
Explanation WITH costs
The Daily Show | Likes : 16634
History | Likes : 19035
The Animal Rescue Site | Likes : 5664
Michelle Obama | Likes : 9722
The Rachel Maddow Show | Likes : 2403
Big Prize Giveaways | Likes : 5833
Mad Men | Likes : 5092
I dont care about your farm, or your fish, or your park, or your mafia!!! | Likes : 8992
*************************************************
ID:  336
Explanation WITHOUT costs
Stevie Ray Vaughn | Likes : 1281
---
Explanation WITH costs
Eric Clapton | Likes : 10373
Stevie Ray Vaughn | Lik

Grey's Anatomy | Likes : 28821
Country music | Likes : 14324
I know alot more than you think I know. I just keep it to myself to save the trouble of an argument. | Likes : 1153
*************************************************
ID:  381
Explanation WITHOUT costs
Peter Frampton | Likes : 1581
Let's remember those who have passed. Press Like if you've lost a loved one... | Likes : 2248
---
Explanation WITH costs
History | Likes : 19035
Eric Clapton | Likes : 10373
Queen | Likes : 19490
NCIS | Likes : 34288
Avatar | Likes : 14592
*************************************************
ID:  382
Explanation WITHOUT costs
Smiling at the bitch who hates you, just because it makes her hate you even more :P | Likes : 1034
---
Explanation WITH costs
Queen | Likes : 19490
*************************************************
ID:  383
Explanation WITHOUT costs
Sewing | Likes : 1549
---
Explanation WITH costs
Bones | Likes : 19370
NCIS | Likes : 34288
*************************************************
ID:  384

ShopAtHome.com | Likes : 1475
Photo of the Day | Likes : 1113
.......And you wonder why i don't talk to you. | Likes : 2466
Click like if you love your daughter! | Likes : 2052
Banners For Profiles | Likes : 1739
---
Explanation WITH costs
Snoop Dogg | Likes : 11950
Michelle Obama | Likes : 9722
Democrats | Likes : 1656
Keyshia Cole | Likes : 7004
Luther Vandross | Likes : 1105
Photo of the Day | Likes : 1113
.......And you wonder why i don't talk to you. | Likes : 2466
Dont judge me on what people have told you, get to know me first. | Likes : 3918
Traveling | Likes : 12693
*************************************************
ID:  426
Explanation WITHOUT costs
Oldies | Likes : 2884
The Animal Rescue Site | Likes : 5664
Six Degrees Of Separation - The Experiment | Likes : 1275
Big Prize Giveaways | Likes : 5833
Wimp.com | Likes : 1907
Dalai Lama | Likes : 5846
RightChange | Likes : 2427
Help Cure Cancer (no money donations needed, only clicks) | Likes : 1381
God understands our prayers ev

Explanation WITH costs
Reading | Likes : 47288
*************************************************
ID:  470
Explanation WITHOUT costs
Tabasco | Likes : 1434
---
Explanation WITH costs
Eric Clapton | Likes : 10373
iTunes | Likes : 20556
*************************************************
ID:  471
Explanation WITHOUT costs
Star Trek | Likes : 11683
Proud to be an American | Likes : 3938
---
Explanation WITH costs
Star Trek | Likes : 11683
Two and a Half Men | Likes : 28055
NCIS | Likes : 34288
*************************************************
ID:  472
Explanation WITHOUT costs
Fargo | Likes : 1799
Snoop Dogg | Likes : 11950
---
Explanation WITH costs
Snoop Dogg | Likes : 11950
Queen | Likes : 19490
*************************************************
ID:  473
Explanation WITHOUT costs
Peter Frampton | Likes : 1581
Bob Seger | Likes : 1739
Rod Stewart | Likes : 1808
---
Explanation WITH costs
Peter Frampton | Likes : 1581
ELVIS PRESLEY | Likes : 6531
Eric Clapton | Likes : 10373
NCIS | Likes : 3

In [37]:
explain.explain(X_sample[13, :], thresholds=t, max_ite=200, stop_at_first=True, cost_func=likes_cost_func)

[[[337, 2092, 3813]]]

Empirical Case 1: Explaining decisions rather than predictions


Empirical Case 2: High-dimensional and context specific explanations

Probabilistic approaches are unreliable when we are dealing with high dimensional spaces. Someone old got targeted with teen stuff, wants to know why. One could show the person the top most informative sites that led to the decision. Turns out those sites are very sensible to the probabilistic procedure. Not a single site appears every time. 

Systematic analysis. Make four groups of people (only people with at least 15 likes), each one for a quantile. For each group compute SHAP values 5 times. For each observation, count the number of pages that appear in all 5 SHAP values. Get percentage for each (0, 1, 2, 3, 4, and 5 matches). Also record the time it took to compute explanations. 

As the number of likes increases, it becomes increasingly unlikely to get consistent results, even if the goal is to find only the top most informative attributes rather than estimating precise weights. 

Our case, an explanation can be found very fast. Do the same analysis showing the average length of explanation for each group.

Problem is that, there are so many features that some of them may not 

(EXAMPLE 1: Facebook Likes? Predict if someone is young.)
- Analysis for fast (vs precise) explanations (case 2). Look for young (or old) person with many likes. Compute SHAP several times to get a distribution of the values. Get average computation time. Compare to computation of EDC (faster, consistent)
- Analysis for "cheapest" explanations (case 3). Show explanations (many likes) when not sorting by popularity. Compare when sorting by popularity.

(EXAMPLE 2: Churn data)
- Incorporate decisions from multiple models. Predict monthly charge and whether the person will churn. Idea: Look for guy where top regression feature and top probability feature are different from top combined feature. Again, show how several features influence expected loss, but only a few are necessary to change decision.